In [3]:
import numpy as np
import imutils


color = (255,255,255)
# defined lower and upper boundaries for each color
# Strongly recommended finding for your own camera.
colors = {'blue': [np.array([95, 255, 85]), np.array([120, 255, 255])],
          'red': [np.array([161, 165, 127]), np.array([178, 255, 255])],
          'yellow': [np.array([16, 0, 99]), np.array([39, 255, 255])],
          'green': [np.array([33, 19, 105]), np.array([77, 255, 255])]}
trajectory = []


import pyrealsense2 as rs
import numpy as np
import cv2

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

if device_product_line == 'L500':
    config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 30)
else:
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

In [4]:

def get_frames():
    # Wait for a coherent pair of frames: depth and color
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    
    # Convert images to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())
    
    # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

    depth_colormap_dim = depth_colormap.shape
    color_colormap_dim = color_image.shape

    # If depth and color resolutions are different, resize color image to match depth image for display
    if depth_colormap_dim != color_colormap_dim:
        resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
        images = np.hstack((resized_color_image, depth_colormap))
    else:
        images = np.hstack((color_image, depth_colormap))

    
    return depth_image, color_image

def compute_contours(frame):
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV) #convertion to HSV
    for name, clr in colors.items(): # for each color in colors
        if find_color(hsv, clr):  # call find_color function above
            c, cx, cy, area = find_color(hsv, clr)
            compute_trajectory(frame, cx, cy)
            compute_spill(frame, cx, cy, area)
            cv.drawContours(frame, [c], -1, color, 3) #draw contours
            cv.circle(frame, (cx, cy), 7, color, -1)  # draw circle
            
            plot_trajectory(frame)
                
            cv.putText(frame, name, (cx,cy), 
                        cv.FONT_HERSHEY_SIMPLEX, 1, color, 1) # put text


def find_color(frame, points):
    mask = cv.inRange(frame, points[0], points[1]) #create mask with boundaries 
    cnts = cv.findContours(mask, cv.RETR_TREE, 
                           cv.CHAIN_APPROX_SIMPLE) # find contours from mask
    cnts = imutils.grab_contours(cnts)
    for c in cnts:
        area = cv.contourArea(c)    # find how big countour is
        if area > 5000:             # only if countour is big enough, then
            M = cv.moments(c)
            cx = int(M['m10'] / M['m00']) # calculate X position
            cy = int(M['m01'] / M['m00']) # calculate Y position
            return c, cx, cy, area
        
def compute_trajectory(frame, cx, cy):
    trajectory.append([cx, cy])
    if(len(trajectory) > 50):
        trajectory.pop(0)
        
def plot_trajectory(frame):
    for point in trajectory:
        cv.circle(frame, (point[0], point[1]), radius=1, color=(255, 0, 0), thickness=-1)
        
        
def compute_spill(frame, cx, cy, area):
    pass

In [5]:
import cv2 as cv

def webcam():
    cap = cv.VideoCapture(0)

    while cap.isOpened(): 
        _, frame = cap.read()
        compute_contours(frame)  
        cv.imshow("Frame: ", frame)
        if cv.waitKey(1) == ord('q'):
            break

    cap.release() 
    cv.destroyAllWindows()


def realsense():
    try:
        pipeline.start(config)
        while True:
            depth_image, color_image = get_frames()
            frame = color_image
            compute_contours(frame)            
            cv.imshow("Realsense Camera Feed", frame)
        
            if cv.waitKey(1) == ord('q'):
                break

    finally:
        pipeline.stop()
    

In [6]:
realsense()